# Máquina de Boltzmann restrita 

## Um exemplo prático: Sistema de Recomendação de Filmes

O código abaixo foi baseado em https://github.com/echen/restricted-boltzmann-machines

### Introdução

Suponha que você peça a vários usuários para classificar um conjunto de filmes em uma escala de 0 a 100. Na análise fatorial clássica , você poderia tentar explicar cada filme e usuário em termos de um conjunto de fatores latentes. Por exemplo, filmes como Guerra nas Estrelas e Senhor dos Anéis podem ter fortes associações com de ficção científica e fantasia, e usuários que gostam de Wall-E e Toy Story podem ter fortes associações com da Pixar.

As máquinas de Boltzmann restritas executam essencialmente uma versão binária da análise fatorial.  Em vez de usuários classificarem um conjunto de filmes de forma contínua escala, eles simplesmente dizem se gostam ou não de um filme, e a RBM tentará descobrir fatores latentes que possam explicar a ativação dessas escolhas de filmes.

Mais tecnicamente, uma máquina de Boltzmann restrita é uma rede neural estocástica ( rede neural significa que temos unidades semelhantes a neurônios cujas ativações binárias dependem dos vizinhos aos quais estão conectadas; estocástica significando que essas ativações têm um elemento probabilístico) consistindo em:

- Uma camada de unidades visíveis (preferências de filmes dos usuários cujos estados conhecemos e definimos);
- Uma camada de unidades ocultas (os fatores latentes que tentamos aprender); e
- Uma unidade de bias (cujo estado está sempre ligado e é uma forma de se ajustar às diferentes popularidades inerentes a cada filme).

Além disso, cada unidade visível é conectada a todas as unidades ocultas (essa conexão não é direcionada, portanto, cada unidade oculta também é conectada a todas as unidades visíveis), e a unidade de polarização é conectada a todas as unidades visíveis e a todas as unidades ocultas. Para facilitar o aprendizado, restringimos a rede para que nenhuma unidade visível seja conectada a qualquer outra unidade visível e nenhuma unidade oculta seja conectada a qualquer outra unidade oculta.

Por exemplo, suponha que temos um conjunto de seis filmes (Harry Potter, Avatar, SdA 3, Gladiador, Titanic e Glitter) e pedimos aos usuários que nos digam quais eles querem assistir. Se quisermos aprender duas unidades latentes subjacentes às preferências de filmes -- por exemplo, dois grupos naturais em nosso conjunto de seis filmes parecem ser ficção científica/fantasia (contendo Harry Potter, Avatar e SdA 3) e vencedores do Oscar (contendo SdA 3, Gladiator e Titanic), então podemos esperar que nossas unidades latentes correspondam a essas categorias - então nosso RBM ficaria assim:

Exemplo de RBM
(Observe a semelhança com um modelo gráfico de análise fatorial.)

### Ativação de estado

Máquinas de Boltzmann restritas e redes neurais em geral funcionam atualizando os estados de alguns neurônios dados os estados de outros, então vamos falar sobre como os estados de unidades individuais mudam. Supondo que conhecemos os pesos de conexão em nosso RBM (explicaremos como aprendê-los abaixo), para atualizar o estado da unidade *i*:

* Calcular a **energia de ativação** $a_i = \sum_j w_{ij} x_j$ da unidade $i$, onde a soma percorre todas as unidades $j$ às quais a unidade $i$ está conectada, $w_{ij} $ é o peso da conexão entre $i$ e $j$, e $x_j$ é o estado 0 ou 1 da unidade $j$. Em outras palavras, todos os vizinhos da unidade $i$ enviam uma mensagem, e nós computamos a soma de todas essas mensagens.
* Seja $p_i = \sigma(a_i)$, onde $\sigma(x) = 1/(1 + exp(-x))$ é a função logística. Observe que $p_i$ está próximo de 1 para grandes energias de ativação positivas e $p_i$ está próximo de 0 para energias de ativação negativas.
* Em seguida, ligamos a unidade $i$ com probabilidade $p_i$ e desligamos com probabilidade $1 - p_i$.
* (Em termos leigos, unidades que estão conectadas positivamente umas com as outras tentam fazer com que umas às outras compartilhem o mesmo estado (ou seja, estejam ligadas ou desligadas), enquanto as unidades que estão conectadas negativamente umas às outras são inimigos que preferem estar em estados diferentes.)

Por exemplo, vamos supor que nossas duas unidades ocultas realmente correspondam a SF/fantasia e vencedores do Oscar.

* Se Alice nos dissesse suas seis preferências binárias em nosso conjunto de filmes, poderíamos então perguntar ao nosso RBM qual das unidades ocultas suas preferências ativam (ou seja, pedir ao RBM para explicar suas preferências em termos de fatores latentes). Assim, os seis filmes enviam mensagens para as unidades ocultas, dizendo-lhes para se atualizarem. (Observe que, mesmo que Alice tenha declarado que quer assistir Harry Potter, Avatar e SdA 3, isso não garante que a unidade oculta de ficção científica/fantasia será ligada, mas apenas que ela será ligada com alta *probabilidade*. Isso faz um pouco de sentido: no mundo real, Alice querendo assistir todos os três filmes nos faz suspeitar que ela gosta de ficção científica/fantasia em geral, mas há uma pequena chance de que ela queira assisti-los por outros motivos. O RBM nos permite *gerar* modelos de pessoas no mundo real e confuso.)
* Por outro lado, se soubermos que uma pessoa gosta de SF/fantasy (de modo que a unidade SF/fantasy esteja ligada), podemos perguntar ao RBM qual das unidades de filme aquela unidade oculta ativa (ou seja, pedir ao RBM para gerar um conjunto de recomendações de filmes). Assim, as unidades ocultas enviam mensagens para as unidades de filme, dizendo-lhes para atualizar seus estados. (Novamente, observe que a unidade SF/fantasy ligada não garante que sempre recomendamos os três Harry Potter, Avatar e LOTR 3 porque, ei, nem todo mundo que gosta de ficção científica gosta de Avatar.)

### Pesos de aprendizagem

Então, como aprendemos os pesos de conexão em nossa rede? Suponha que temos vários exemplos de treinamento, onde cada exemplo de treinamento é um vetor binário com seis elementos correspondentes às preferências de filme de um usuário. Em seguida, para cada época, faça o seguinte:

* Veja um exemplo de treinamento (um conjunto de seis preferências de filme). Defina os estados das unidades visíveis para essas preferências.
* Em seguida, atualize os estados das unidades ocultas usando a regra de ativação logística descrita acima: para a $j$th unidade oculta, calcule sua energia de ativação $a_j = \sum_i w_{ij} x_i$ e defina $x_j$ como 1 com probabilidade $\sigma(a_j)$ e para 0 com probabilidade $1 - \sigma(a_j)$. Então, para cada aresta $e_{ij}$, calcule $Positive(e_{ij}) = x_i * x_j$ (ou seja, para cada par de unidades, meça se ambas estão ligadas).
* Agora **reconstrua** as unidades visíveis de maneira semelhante: para cada unidade visível, calcule sua energia de ativação $a_i$ e atualize seu estado. (Observe que essa *reconstrução* pode não corresponder às preferências originais.) Em seguida, atualize as unidades ocultas novamente e calcule $Negative(e_{ij}) = x_i * x_j$ para cada aresta.
* Atualize o peso de cada aresta $e_{ij}$ definindo $w_{ij} = w_{ij} + L * (Positive(e_{ij}) - Negative(e_{ij}))$, onde $L $ é uma taxa de aprendizado.
* Repita em todos os exemplos de treinamento.

Continue até que a rede convirja (ou seja, o erro entre os exemplos de treinamento e suas reconstruções caia abaixo de algum limite) ou alcancemos um número máximo de épocas.

Por que essa regra de atualização faz sentido? Observe que

* Na primeira fase, $Positive(e_{ij})$ mede a associação entre a unidade $i$th e $j$th que *queremos* que a rede aprenda com nossos exemplos de treinamento;
* Na fase de "reconstrução", onde o RBM gera os estados das unidades visíveis com base em suas hipóteses sobre as unidades ocultas apenas, $Negative(e_{ij})$ mede a associação que a própria rede *própria* gera (ou "sonhos acordados" " about) quando nenhuma unidade é fixada aos dados de treinamento.

Então, ao adicionar $Positive(e_{ij}) - Negative(e_{ij})$ a cada peso de aresta, estamos ajudando os devaneios da rede a corresponderem melhor à realidade de nossos exemplos de treinamento.

(Você pode ouvir esta regra de atualização chamada **divergência contrastiva**, que é basicamente um termo estranho para "descida de gradiente aproximada".)



### Exemplo

Primeiro, treinei o RBM usando alguns dados falsos.

* Alice: (Harry Potter = 1, Avatar = 1, SdA 3 = 1, Gladiador = 0, Titanic = 0, Glitter = 0). Grande fã de ficção científica/fantasia.
* Bob: (Harry Potter = 1, Avatar = 0, SdA 3 = 1, Gladiador = 0, Titanic = 0, Glitter = 0). Fã de ficção científica/fantasia, mas não gosta de Avatar.
* Carol: (Harry Potter = 1, Avatar = 1, SdA 3 = 1, Gladiador = 0, Titanic = 0, Glitter = 0). Grande fã de ficção científica/fantasia.
* David: (Harry Potter = 0, Avatar = 0, SdA 3 = 1, Gladiador = 1, Titanic = 1, Glitter = 0). Fã de grandes vencedores do Oscar.
* Eric: (Harry Potter = 0, Avatar = 0, SdA 3 = 1, Gladiador = 1, Titanic = 1, Glitter = 0). Fã de vencedores do Oscar, exceto Titanic.
* Fred: (Harry Potter = 0, Avatar = 0, SdA 3 = 1, Gladiador = 1, Titanic = 1, Glitter = 0). Fã de grandes vencedores do Oscar.

A rede aprendeu os seguintes pesos:

    Unidade de polarização oculta 1 oculta 2
    Unidade de Viés -0,08257658 -0,19041546 1,57007782
    Harry Potter -0,82602559 -7,08986885 4,96606654
    Avatar -1.84023877 -5.18354129 2.27197472
    LOTR 3 3.92321075 2.51720193 4.11061383
    Gladiador 0,10316995 6,74833901 -4,00505343
    Titanic -0,97646029 3,25474524 -5,59606865
    Glitter -4.44685751 -2.81563804 -2.91540988


Observe que a primeira unidade oculta parece corresponder aos vencedores do Oscar, e a segunda unidade oculta parece corresponder aos filmes de ficção científica/fantasia, exatamente como esperávamos.

O que acontece se dermos ao RBM um novo usuário, George, que tem (Harry Potter = 0, Avatar = 0, LOTR 3 = 0, Gladiator = 1, Titanic = 1, Glitter = 0) como suas preferências? Ele liga a unidade dos vencedores do Oscar (mas não a unidade de ficção científica/fantasia), adivinhando corretamente que George provavelmente gosta de filmes vencedores do Oscar.

O que acontece se ativarmos apenas a unidade SF/fantasy e executarmos o RBM várias vezes? Nos meus testes, ele ligou Harry Potter, Avatar e LOTR 3 três vezes; ele ativou Avatar e LOTR 3, mas não Harry Potter, uma vez; e ligou Harry Potter e LOTR 3, mas não Avatar, duas vezes. Observe que, com base em nossos exemplos de treinamento, essas preferências geradas realmente correspondem ao que esperamos que os fãs reais de ficção científica/fantasia queiram assistir.

### Modificações 

(O código abaixo foi baseado em https://github.com/echen/restricted-boltzmann-machines)

Tentei manter o algoritmo de aprendizado de conexão que descrevi acima bem simples, então aqui estão algumas modificações que geralmente aparecem na prática:

* Acima, $Negative(e_{ij})$ foi determinado tomando o produto das unidades $i$th e $j$th após reconstruir as unidades visíveis *uma vez* e então atualizar as unidades ocultas novamente. Também poderíamos obter o produto após um número maior de reconstruções (ou seja, repetir a atualização das unidades visíveis, depois as unidades ocultas, depois as unidades visíveis novamente e assim por diante); isso é mais lento, mas descreve os devaneios da rede com mais precisão.
* Em vez de usar $Positive(e_{ij})=x_i * x_j$, onde $x_i$ e $x_j$ são binários 0 ou 1 *estados*, também podemos deixar $x_i$ e/ou $x_j$ como ativação *probabilidades*. Da mesma forma para $Negative(e_{ij})$.
* Poderíamos penalizar pesos de aresta maiores, a fim de obter um modelo mais esparso ou mais regularizado.
* Ao atualizar os pesos das arestas, poderíamos usar um fator de momento: adicionaríamos a cada aresta uma soma ponderada da etapa atual conforme descrito acima (ou seja, $L * (Positive(e_{ij}) - Negative(e_{ij}) )$) e o passo dado anteriormente.
* Em vez de usar apenas um exemplo de treinamento em cada época, poderíamos usar *lotes* de exemplos em cada época, e só atualizar os pesos da rede após passar por todos os exemplos do lote. Isso pode acelerar o aprendizado aproveitando os algoritmos de multiplicação de matrizes rápidos.

# **Vamos começar!**

O código abaixo foi baseado em https://github.com/echen/restricted-boltzmann-machines



In [4]:
from __future__ import print_function
import numpy as np

class RBM:
  
  def __init__(self, num_visible, num_hidden):
    self.num_hidden = num_hidden
    self.num_visible = num_visible
    self.debug_print = True

    # Inicializa uma matriz de pesos, de dimensões (num_visible x num_hidden), usando
    # uma distribuição uniforme entre -sqrt(6. / (num_hidden + num_visible))
    # e sqrt(6. / (num_hidden + num_visible)). Pode-se variar o
    # desvio padrão multiplicando o intervalo pelo valor apropriado.
    # Aqui inicializamos os pesos com média 0 e desvio padrão 0,1.
    # Referência: Understanding the difficulty of training deep feedforward 
    # neural networks by Xavier Glorot and Yoshua Bengio
    
    np_rng = np.random.RandomState(1234)

    self.weights = np.asarray(np_rng.uniform(
			low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	size=(num_visible, num_hidden)))


    # Insira pesos para as unidades de polarização na primeira linha e na primeira coluna.
    self.weights = np.insert(self.weights, 0, 0, axis = 0)
    self.weights = np.insert(self.weights, 0, 0, axis = 1)

  def train(self, data, max_epochs = 1000, learning_rate = 0.1):
    """
    Treine a máquina.
    Parâmetros
    ----------
    data: Uma matriz onde cada linha é um exemplo de treinamento que consiste nos estados das unidades visíveis.     
    """

    num_examples = data.shape[0]

    # Insira unidades de polarização de 1 na primeira coluna.
    data = np.insert(data, 0, 1, axis = 1)

    for epoch in range(max_epochs):      
      # Fixe os dados e a amostra das unidades ocultas.
      # (Esta é a "fase positiva do CD", também conhecida como a fase da realidade.)
      pos_hidden_activations = np.dot(data, self.weights)      
      pos_hidden_probs = self._logistic(pos_hidden_activations)
      pos_hidden_probs[:,0] = 1 # Corrige a unidade de polarização.
      pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
      # Observe que estamos usando as *probabilidades* de ativação dos estados ocultos, não os estados ocultos       
      # eles mesmos, ao computar associações. Também poderíamos usar os estados; veja a seção 3 de Hinton's
      #"A Practical Guide to Training Restricted Boltzmann Machines"  para mais informações.
      pos_associations = np.dot(data.T, pos_hidden_probs)

      # Reconstruir as unidades visíveis e amostrar novamente das unidades ocultas.
      # (Esta é a "fase negativa do CD", também conhecida como fase de devaneio.)
      neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
      neg_visible_probs = self._logistic(neg_visible_activations)
      neg_visible_probs[:,0] = 1 # Corrige a unidade de polarização.
      neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
      neg_hidden_probs = self._logistic(neg_hidden_activations)
      # Observe, novamente, que estamos usando as *probabilidades* de ativação ao calcular associações, não os estados
      # eles mesmos.
      neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

      # Atualizar pesos.
      self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)

      error = np.sum((data - neg_visible_probs) ** 2)
      if self.debug_print:
        print("Época %s: erro é %s"  % (epoch, error))

  def run_visible(self, data):
    """
    Assumindo que o RBM foi treinado (de modo que os pesos para a rede foram aprendidos),
    execute a rede em um conjunto de unidades visíveis, para obter uma amostra das unidades ocultas.
    
    Parâmetros
    ----------
    data: Uma matriz onde cada linha consiste nos estados das unidades visíveis.
    
    Devoluções
    -------
    estados_escondidos: Uma matriz onde cada linha consiste nas unidades ocultas ativadas a partir do
    unidades na matriz de dados passada.
    """
    
    num_examples = data.shape[0]
    
    # Crie uma matriz, onde cada linha deve ser as unidades ocultas (mais uma unidade de polarização)
    # amostrado de um exemplo de treinamento.
    hidden_states = np.ones((num_examples, self.num_hidden + 1))
    
    # Insira unidades de polarização de 1 na primeira coluna de dados.
    data = np.insert(data, 0, 1, axis = 1)

    # Calcular as ativações das unidades ocultas.
    hidden_activations = np.dot(data, self.weights)
    # Calcular as probabilidades de ativar as unidades ocultas.
    hidden_probs = self._logistic(hidden_activations)
    # Ligue as unidades ocultas com suas probabilidades especificadas.
    hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
    # Sempre fixe a unidade de polarização em 1.
    # estados_escondidos[:,0] = 1
  
    # Ignora as unidades de polarização.
    hidden_states = hidden_states[:,1:]
    return hidden_states
    
  # TODO: Remover a duplicação de código entre este método e `run_visible`?
  def run_hidden(self, data):
    """
    Assumindo que o RBM foi treinado (de modo que os pesos para a rede foram aprendidos),
    execute a rede em um conjunto de unidades ocultas, para obter uma amostra das unidades visíveis.
    Parâmetros
    ----------
    dados: Uma matriz onde cada linha consiste nos estados das unidades ocultas.
    Devoluções
    -------
    visible_states: Uma matriz onde cada linha consiste nas unidades visíveis ativadas do oculto
    unidades na matriz de dados passada.
    """

    num_examples = data.shape[0]

    # Crie uma matriz, onde cada linha deve ser as unidades visíveis (mais uma unidade de polarização)
    # amostrado de um exemplo de treinamento.
    visible_states = np.ones((num_examples, self.num_visible + 1))

    # Insira unidades de polarização de 1 na primeira coluna de dados.
    data = np.insert(data, 0, 1, axis = 1)

    # Calcular as ativações das unidades visíveis.
    visible_activations = np.dot(data, self.weights.T)
    # Calcular as probabilidades de ligar as unidades visíveis.
    visible_probs = self._logistic(visible_activations)
    # Ligue as unidades visíveis com suas probabilidades especificadas.
    visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
    # Sempre fixe a unidade de polarização em 1.
    # visible_states[:,0] = 1

    # Ignora as unidades de polarização.
    visible_states = visible_states[:,1:]
    return visible_states
    
  def daydream(self, num_samples):
    """
    Inicialize aleatoriamente as unidades visíveis uma vez e comece a executar etapas de amostragem Gibbs alternadas
    (onde cada etapa consiste em atualizar todas as unidades ocultas e, em seguida, atualizar todas as unidades visíveis),
    tomando uma amostra das unidades visíveis em cada etapa.
    Observe que inicializamos a rede apenas *uma vez*, portanto, essas amostras são correlacionadas.
    Devoluções
    -------
    amostras: Uma matriz, onde cada linha é uma amostra das unidades visíveis produzidas enquanto a rede foi
    daydream
    """

   # Crie uma matriz, onde cada linha deve ser uma amostra das unidades visíveis
    # (com uma unidade de polarização extra), inicializado para todos.
    samples = np.ones((num_samples, self.num_visible + 1))

    # Pegue a primeira amostra de uma distribuição uniforme.
    samples[0,1:] = np.random.rand(self.num_visible)

    # Inicie a amostragem Gibbs alternada.
    # Observe que mantemos os estados binários das unidades ocultas, mas deixamos o
    # unidades visíveis como probabilidades reais. Veja a seção 3 do Hinton's
    # "A Practical Guide to Training Restricted Boltzmann Machines"
    # para saber mais sobre o porquê.
    for i in range(1, num_samples):
      visible = samples[i-1,:]

      # Calcular as ativações das unidades ocultas.
      hidden_activations = np.dot(visible, self.weights)      
      # Calcular as probabilidades de ativar as unidades ocultas.
      hidden_probs = self._logistic(hidden_activations)
      # Ligue as unidades ocultas com suas probabilidades especificadas.
      hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
      # Sempre fixe a unidade de polarização em 1.
      hidden_states[0] = 1

      # Recalcular as probabilidades de que as unidades visíveis estejam ativadas.
      visible_activations = np.dot(hidden_states, self.weights.T)
      visible_probs = self._logistic(visible_activations)
      visible_states = visible_probs > np.random.rand(self.num_visible + 1)
      samples[i,:] = visible_states

    # Ignore as unidades de polarização (a primeira coluna), pois elas sempre são definidas como 1.
    return samples[:,1:]        
      
  def _logistic(self, x):
    return 1.0 / (1 + np.exp(-x))

if __name__ == '__main__':
  r = RBM(num_visible = 6, num_hidden = 2)
  training_data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])
  r.train(training_data, max_epochs = 5000)
  print(r.weights)
  user = np.array([[0,0,0,1,1,0]])
  print(r.run_visible(user))


Época 0: erro é 9.01918227204572
Época 1: erro é 8.775609522743041
Época 2: erro é 8.56943871904196
Época 3: erro é 8.413047930939022
Época 4: erro é 8.17124173259156
Época 5: erro é 7.936459557145244
Época 6: erro é 7.961247745467006
Época 7: erro é 7.802289811873741
Época 8: erro é 7.825485785761026
Época 9: erro é 7.343104191931459
Época 10: erro é 7.41744048755596
Época 11: erro é 7.044371417285086
Época 12: erro é 6.967055635386924
Época 13: erro é 7.167276431878502
Época 14: erro é 6.907517242244686
Época 15: erro é 6.732897061994529
Época 16: erro é 6.876717232060717
Época 17: erro é 6.796206710718401
Época 18: erro é 6.746415620733424
Época 19: erro é 6.822749667691085
Época 20: erro é 6.836187194251136
Época 21: erro é 6.4939684324161995
Época 22: erro é 6.698085793913448
Época 23: erro é 6.535612114554365
Época 24: erro é 6.4813927498857336
Época 25: erro é 6.264346729583171
Época 26: erro é 6.328568385419738
Época 27: erro é 6.622724687986897
Época 28: erro é 6.1762933277794

Época 754: erro é 1.3613947660930974
Época 755: erro é 1.3610834583146358
Época 756: erro é 1.3607837198429873
Época 757: erro é 1.3604951017651166
Época 758: erro é 1.3602171703756414
Época 759: erro é 1.3599495068261163
Época 760: erro é 1.359691706770272
Época 761: erro é 1.359443380006409
Época 762: erro é 1.1664838943536218
Época 763: erro é 3.1339032463480136
Época 764: erro é 1.3609425523462009
Época 765: erro é 1.145343350674431
Época 766: erro é 1.3613624073630222
Época 767: erro é 1.3610338054156244
Época 768: erro é 1.3607172482109786
Época 769: erro é 1.3604122815282804
Época 770: erro é 1.3601184657876473
Época 771: erro é 1.1442181210852118
Época 772: erro é 1.3605407015278992
Época 773: erro é 1.3602324753896755
Época 774: erro é 1.3599355393395656
Época 775: erro é 1.3596494641666865
Época 776: erro é 1.3593738347700928
Época 777: erro é 1.3591082498627616
Época 778: erro é 1.3588523216690438
Época 779: erro é 1.3586056756168796
Época 780: erro é 1.1462072668990657
Époc

Época 1338: erro é 1.35002324355349
Época 1339: erro é 1.3495304417356906
Época 1340: erro é 1.3490562625470441
Época 1341: erro é 1.0133826250817308
Época 1342: erro é 1.349772516920035
Época 1343: erro é 1.0091539954973319
Época 1344: erro é 1.350484411183866
Época 1345: erro é 1.3499743366885566
Época 1346: erro é 1.349483393533869
Época 1347: erro é 1.00764709029947
Época 1348: erro é 1.002263245854502
Época 1349: erro é 1.537112037295093
Época 1350: erro é 1.35254929532733
Época 1351: erro é 0.9999646479925075
Época 1352: erro é 0.9948451685685807
Época 1353: erro é 1.3545019691441376
Época 1354: erro é 1.3538389266145008
Época 1355: erro é 1.3531996965598527
Época 1356: erro é 1.3525835794895722
Época 1357: erro é 1.3519898870386275
Época 1358: erro é 1.8878393726791989
Época 1359: erro é 1.3508246218636268
Época 1360: erro é 1.0063139865452706
Época 1361: erro é 1.3515017142991965
Época 1362: erro é 1.3509415557570181
Época 1363: erro é 1.3504022510716682
Época 1364: erro é 1.69

Época 2026: erro é 1.364066156262217
Época 2027: erro é 0.839185279449718
Época 2028: erro é 1.364583954193878
Época 2029: erro é 0.8380069031825214
Época 2030: erro é 1.3650892564918742
Época 2031: erro é 0.8368626990038522
Época 2032: erro é 0.8341650011712793
Época 2033: erro é 0.8315488180675185
Época 2034: erro é 1.3686547069509778
Época 2035: erro é 1.3675137994257125
Época 2036: erro é 1.366407950179404
Época 2037: erro é 1.3653364807614665
Época 2038: erro é 1.3642987033328402
Época 2039: erro é 0.8369745602762884
Época 2040: erro é 1.3648137348027387
Época 2041: erro é 0.8358221247647066
Época 2042: erro é 1.36531627275298
Época 2043: erro é 0.8347030718292978
Época 2044: erro é 1.3658065144792928
Época 2045: erro é 0.833616038018092
Época 2046: erro é 1.3662846660465284
Época 2047: erro é 0.8325597291155559
Época 2048: erro é 2.344782121049961
Época 2049: erro é 0.8342677939901197
Época 2050: erro é 0.831633023495804
Época 2051: erro é 1.368495301687477
Época 2052: erro é 0.8

Época 2742: erro é 0.7038759086530895
Época 2743: erro é 0.7036650441233111
Época 2744: erro é 1.496873724153268
Época 2745: erro é 0.7042383082282647
Época 2746: erro é 1.4946472419432053
Época 2747: erro é 0.7048258637222091
Época 2748: erro é 0.704594741337382
Época 2749: erro é 0.7043674768224554
Época 2750: erro é 0.7041439855064282
Época 2751: erro é 1.624816803047628
Época 2752: erro é 0.7027434256581732
Época 2753: erro é 0.7025485365926464
Época 2754: erro é 0.7023567225570497
Época 2755: erro é 0.70216791904925
Época 2756: erro é 1.5014228376518095
Época 2757: erro é 0.7027163526485524
Época 2758: erro é 0.7025195764049909
Época 2759: erro é 0.702325918307008
Época 2760: erro é 0.702135312762448
Época 2761: erro é 0.7019476958290032
Época 2762: erro é 0.7017630051656026
Época 2763: erro é 0.7015811799854398
Época 2764: erro é 0.7014021610105746
Época 2765: erro é 1.5035952542141726
Época 2766: erro é 1.500670979832234
Época 2767: erro é 1.4977456296202603
Época 2768: erro é 0

Época 3406: erro é 0.6819029571840454
Época 3407: erro é 0.6818728390433404
Época 3408: erro é 0.6818429630461713
Época 3409: erro é 0.6818133261082276
Época 3410: erro é 1.6069767536162072
Época 3411: erro é 0.681510892737604
Época 3412: erro é 1.5692553526954551
Época 3413: erro é 0.6816997188905485
Época 3414: erro é 0.68167120532661
Época 3415: erro é 0.6816429135862899
Época 3416: erro é 0.6816148408859979
Época 3417: erro é 0.6815869844829212
Época 3418: erro é 0.6815593416743432
Época 3419: erro é 1.568045260053494
Época 3420: erro é 0.6817594620054358
Época 3421: erro é 0.6817294738356457
Época 3422: erro é 0.6816997271255147
Época 3423: erro é 0.6816702188131499
Época 3424: erro é 0.6816409458820104
Época 3425: erro é 0.6816119053601347
Época 3426: erro é 0.6815830943193837
Época 3427: erro é 0.6815545098746991
Época 3428: erro é 0.681526149183375
Época 3429: erro é 1.5674733038032087
Época 3430: erro é 0.6817328484138717
Época 3431: erro é 0.6817020753209151
Época 3432: erro 

Época 4006: erro é 0.6758432974572124
Época 4007: erro é 0.675833666473625
Época 4008: erro é 0.6758241668524019
Época 4009: erro é 1.6031955831953295
Época 4010: erro é 0.6758323485965199
Época 4011: erro é 0.6758228831739546
Época 4012: erro é 0.6758135316712386
Época 4013: erro é 0.6758042885832697
Época 4014: erro é 1.6016581452004548
Época 4015: erro é 0.6758263670336664
Época 4016: erro é 0.675816987602041
Época 4017: erro é 0.6758077043530555
Época 4018: erro é 0.6757985127987494
Época 4019: erro é 0.6757894086708014
Época 4020: erro é 0.6757803879103708
Época 4021: erro é 1.6002521123867193
Época 4022: erro é 0.6758154046748077
Época 4023: erro é 0.6758060807010002
Época 4024: erro é 0.675796833485778
Época 4025: erro é 0.6757876596892138
Época 4026: erro é 0.6757785561358397
Época 4027: erro é 1.5987854366599425
Época 4028: erro é 0.675826174044376
Época 4029: erro é 0.6758166267849562
Época 4030: erro é 0.6758071469752726
Época 4031: erro é 0.6757977319681976
Época 4032: erro

Época 4509: erro é 0.6740032980143471
Época 4510: erro é 0.6739921243471901
Época 4511: erro é 0.6739810174912818
Época 4512: erro é 0.6739699768551277
Época 4513: erro é 0.6739590018533785
Época 4514: erro é 0.6739480919067479
Época 4515: erro é 0.6739372464419296
Época 4516: erro é 1.5945113913859268
Época 4517: erro é 0.6740663063558376
Época 4518: erro é 0.6740544100875115
Época 4519: erro é 0.6740425874559243
Época 4520: erro é 0.6740308377971388
Época 4521: erro é 0.6740191604543218
Época 4522: erro é 0.6740075547776412
Época 4523: erro é 0.6739960201241677
Época 4524: erro é 0.6739845558577781
Época 4525: erro é 0.6739731613490605
Época 4526: erro é 0.6739618359752214
Época 4527: erro é 0.6739505791199968
Época 4528: erro é 0.6739393901735621
Época 4529: erro é 0.6739282685324456
Época 4530: erro é 0.6739172135994447
Época 4531: erro é 0.6739062247835418
Época 4532: erro é 0.6738953014998238
Época 4533: erro é 0.6738844431694017
Época 4534: erro é 0.6738736492193322
Época 4535: 

In [5]:
import numpy as np
import pandas as pd

## Carregando os dados


In [6]:
base = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
34203,151697,Grand Slam (1967),Thriller
34204,151701,Bloodmoney (2010),(no genres listed)
34205,151703,The Butterfly Circus (2009),Drama
34206,151709,Zero (2015),Drama|Sci-Fi


In [7]:
df = base[['userId', 'movieId', 'rating']][:20000]

df.loc[df.rating <= 2.5, 'rating'] = 0
df.loc[df.rating > 2.5, 'rating'] = 1

df = df.pivot(index='userId', columns='movieId')
df.fillna(0, inplace=True)

users = df[200:218].to_numpy()
df = df[:200].to_numpy()

### Construção do RBM

Primeiro, inicialize um RBM com o número desejado de unidades visíveis e ocultas.

In [8]:
rbm = RBM(num_visible = 4715, num_hidden = 3000)

#### Topologia da Rede

#### Treinamento

In [ ]:
rbm.train(df, max_epochs=10000)
rbm.weights

Época 0: erro é 236133.15566817392
Época 1: erro é 26628.937332743415
Época 2: erro é 26660.903648847503
Época 3: erro é 25977.30059708008
Época 4: erro é 25662.439273780747
Época 5: erro é 25317.27447122814
Época 6: erro é 25172.47713763108
Época 7: erro é 24953.703184965165
Época 8: erro é 24737.130157746225
Época 9: erro é 24631.48756398705
Época 10: erro é 24628.741967700884
Época 11: erro é 24341.475506405233


### RECOMENDAÇÃO


In [ ]:
#Dado um novo conjunto de unidades visíveis, podemos ver quais unidades ocultas estão ativadas.
usuario =  np.array([users[0]])

camada_escondida = rbm.run_visible(usuario)
camada_escondida

In [ ]:
## Dado um conjunto de unidades ocultas, podemos ver quais unidades visíveis estão ativadas.
recomendacao = rbm.run_hidden(camada_escondida)
recomendacao

In [ ]:
for i in range(len(usuario[0])):
    if usuario1[0,i] == 0 and recomendacao[0,i] == 1:
        print(movies['title'][movies['movieId'] ==  base['movieId'][i]].values[0])